In [3]:
import glob
import argparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import utils
import glob
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt 

In [4]:
# Setup oggm: important we want to use oggm's version 62 of all glacier geometries. This command will download them.
# utils.get_rgi_dir(version='62')

In [5]:
path_O1_shp = r'C:\Users\damsg\OGGM\rgi\RGIV62\00_rgi62_regions\00_rgi62_O1Regions.shp' # shp file of regional boundaries
# C:\Users\damsg\OGGM\rgi\RGIV62
# Import 20-bin gridded training dataset
metadata_file = "metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv"
glathida_rgis = pd.read_csv(metadata_file, low_memory=False)

In [6]:
# See all RGI's regional boundaries
# See pag 8 at https://nsidc.org/sites/nsidc.org/files/technical-references/RGI_Tech_Report_V6.0.pdf
world = gpd.read_file(path_O1_shp)      # import as geopandas dataframe
print(world)
fig, ax = plt.subplots()
for n, rgi_series in enumerate(world['geometry']):
    x, y = rgi_series.exterior.xy
    min_x, max_x = min(x), max(x)
    min_y, max_y = min(y), max(y)
    ax.plot(x, y, c='g')
    ax.text(np.mean([min_x, max_x]), np.mean([min_y, max_y]), f"{world['geometry'].iloc[n]}", color='green', fontsize=9)
plt.show()

                     FULL_NAME RGI_CODE WGMS_CODE  \
0                       Alaska        1       ALA   
1                       Alaska        1       ALA   
2       Western Canada and USA        2       WNA   
3         Arctic Canada, North        3       ACN   
4         Arctic Canada, South        4       ACS   
5          Greenland Periphery        5       GRL   
6                      Iceland        6       ISL   
7       Svalbard and Jan Mayen        7       SJM   
8                  Scandinavia        8       SCA   
9               Russian Arctic        9       RUA   
10                 Asia, North       10       ASN   
11                 Asia, North       10       ASN   
12              Central Europe       11       CEU   
13    Caucasus and Middle East       12       CAU   
14               Asia, Central       13       ASC   
15            Asia, South West       14       ASW   
16            Asia, South East       15       ASE   
17               Low Latitudes       16       

In [13]:
# Let's select one region of interest, e.g. rgi=11 (Central Europe)

# Dataframe of glaciers for rgi=11. This dataset contains a lot of info, some of them are also used to create the
# training dataset. RGIId is the official name of glaciers according to the Randolph Glacier Inventory. 
# In the column geometry you get the geometries. 

rgi = 5
# oggm_rgi_shp = glob.glob(f"/home/maffe/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0] # .shp file for rgi 11
# oggm_rgi_shp =  glob.glob(f"C:/Users/damsg/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0]
# oggm_rgi_shp =  glob.glob(f"C:/Users/damsg/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0]
oggm_rgi_shp =  f"C:/Users/damsg/OGGM/rgi/RGIV62/05_rgi62_GreenlandPeriphery/05_rgi62_GreenlandPeriphery.shp"

oggm_rgi_glaciers = gpd.read_file(oggm_rgi_shp)                         # dataframe for rgi 11

In [14]:
# Plot one specific glacier by passing the RGIId code.
# Example: RGI60-11.01450 is the Aletsch Glacier, the biggest in Central Europe (rgi=11).
# You under the geometry column you find the glacier external boundary (.external)
# and the glacier nunataks, if any (.interiors). Nunataks are the inner glacier portions that are deglaciated (=rock).
# We can also find the points in the training dataset for this glacier (not all glaciers contain measurements)

# glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']=='RGI60-11.01450']['geometry'].item()
glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']=='RGI60-05.12964']['geometry'].item()

# Get the measurements in the training dataset by passing the glacier id
glathida_rgis_aletsch = glathida_rgis.loc[glathida_rgis['RGIId']=='RGI60-05.12964']
print(f"We have {len(glathida_rgis_aletsch)} points in the training dataset")

fig, ax = plt.subplots()
exterior_ring = glacier_geometry.exterior # External geometry
ax.plot(*exterior_ring.xy, c='b')
glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors] # Nunataks (list may be empty if no nunataks)
for nunatak in glacier_nunataks_list:
    ax.plot(*nunatak.xy, c='k', lw=0.8)
ax.scatter(x=glathida_rgis_aletsch['POINT_LON'], y=glathida_rgis_aletsch['POINT_LAT'], s=20, c='orange', label='Training points')
ax.legend()
plt.show()

We have 21 points in the training dataset


In [15]:
# Plot the whole regional set of glaciers and all nunataks
# zoom in with matplotlib in interactive mode (%matplotlib qt ) to have a sense of all glacier and their nunataks

region11 = world.loc[5] # Regional boundary
fig, ax = plt.subplots()
for glacier_rgiid, glacier_geometry in zip(oggm_rgi_glaciers['RGIId'], oggm_rgi_glaciers['geometry']):

    exterior_ring = glacier_geometry.exterior
    glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors]

    ax.plot(*exterior_ring.xy, c='b')
    for nunatak in glacier_nunataks_list:
        ax.plot(*nunatak.xy, c='k', lw=0.8) # Plot glacier nunataks (if any)
ax.plot(*region11['geometry'].exterior.xy, c='g') # Regional boundary
ax.text(min(*region11['geometry'].exterior.xy[0])+1,
        max(*region11['geometry'].exterior.xy[1])-1, f'rgi {rgi}', c='g', fontsize=12)
plt.show()

In [16]:
np.unique(glathida_rgis['RGIId'],return_counts=True)

(array(['RGI60-01.08989', 'RGI60-01.10006', 'RGI60-01.10196', ...,
        'RGI60-19.01407', 'RGI60-19.01415', 'RGI60-19.01721'], dtype=object),
 array([ 63,  33,  42, ...,  11, 103,  63], dtype=int64))

In [17]:
np.unique(glathida_rgis['RGI'],return_counts=True)

(array([ 1.,  2.,  3.,  4.,  5.,  7.,  8., 10., 11., 12., 13., 16., 17.,
        19.]),
 array([ 1839,   121, 16148,  8796, 32413,  7147,  2118,   136,  5986,
          223,   880,   139,   668,  4676], dtype=int64))

In [18]:
np.shape(np.unique(glathida_rgis['RGIId'],return_counts=True)[0])

(2321,)

In [19]:
a = glathida_rgis[glathida_rgis['RGIId']=='RGI60-05.12964']
a['THICKNESS']

53638     41.0
53639     56.0
53640     66.5
53641     77.0
53642     89.0
53643     83.0
53644     93.5
53645     96.0
53646    100.0
53647    105.0
53648    109.0
53649     97.5
53650     81.0
53651     59.0
53652     70.0
53653     40.0
53654     19.0
53655      3.5
53656      0.0
53657      0.0
53658      0.0
Name: THICKNESS, dtype: float64

In [20]:
glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']=='RGI60-05.12964']['geometry'].item()


In [21]:

polygon_names = np.unique(glathida_rgis['RGIId'])
#polygon_names[0] = RGI60-01.08989
#The rgi value is found after RGI-60- and before the period
rgi = 5
oggm_rgi_shp =  glob.glob(f"C:/Users/damsg/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0]
# oggm_rgi_shp =  f"C:/Users/damsg/OGGM/rgi/RGIV62/05_rgi62_GreenlandPeriphery/05_rgi62_GreenlandPeriphery.shp"

oggm_rgi_glaciers = gpd.read_file(oggm_rgi_shp) 
glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']==polygon_names]['geometry'].item()


IndexError: list index out of range

In [ ]:
plt.plot(*glacier_geometry.exterior.xy)


In [12]:

# Example polygon_names array
# Replace this with the actual array from your data
polygon_names = np.unique(glathida_rgis['RGIId'])

# Function to extract the RGI value from the RGIId
def extract_rgi_value(rgi_id):
    return rgi_id.split('-')[1].split('.')[0]

# Directory to save the glacier geometries as images
output_dir = "C:/Users/damsg/OGGM/glacier_geometries_images_small/"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

desired_width = 64
desired_height = 64
dpi = 100  # Use a higher DPI to avoid matplotlib rendering issues
import glob
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Example polygon_names array
# Replace this with the actual array from your data
polygon_names = np.unique(glathida_rgis['RGIId'])

# Function to extract the RGI value from the RGIId
def extract_rgi_value(rgi_id):
    return rgi_id.split('-')[1].split('.')[0]

# Directory to save the glacier geometries as images
output_dir = "C:/Users/damsg/OGGM/glacier_geometries_images_small/"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

desired_width = 64
desired_height = 64
dpi = 100  # Use a higher DPI to avoid matplotlib rendering issues

# Cache shapefile paths
shapefile_cache = {}

def get_shapefile_path(rgi):
    if rgi not in shapefile_cache:
        pattern = f"C:/Users/damsg/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp"
        shapefile_cache[rgi] = glob.glob(pattern)[0]
    return shapefile_cache[rgi]

# Iterate over each RGIId in polygon_names
for rgi_id in polygon_names:
    # Extract the RGI value
    rgi = extract_rgi_value(rgi_id)
    
    # Get the shapefile path from the cache
    oggm_rgi_shp = get_shapefile_path(rgi)
    
    # Read the shapefile
    oggm_rgi_glaciers = gpd.read_file(oggm_rgi_shp)
    
    # Extract the geometry for the current RGIId
    glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId'] == rgi_id]['geometry'].item()

    # Create the plot with a larger size and higher DPI
    fig, ax = plt.subplots(figsize=(desired_width / dpi, desired_height / dpi), dpi=dpi)

    # Plot your glacier geometry
    gpd.GeoSeries([glacier_geometry]).plot(ax=ax)

    # Remove axes
    ax.set_axis_off()

    # Save the plot as an image file
    temp_filename = os.path.join(output_dir, f"{rgi_id}_temp.png")
    plt.savefig(temp_filename, pad_inches=0, bbox_inches='tight')

    # Close the plot to free memory
    plt.close()

    # Resize the saved image to 64x64 pixels
    output_filename = os.path.join(output_dir, f"{rgi_id}.png")
    image = Image.open(temp_filename)
    resized_image = image.resize((desired_width, desired_height))
    resized_image.save(output_filename)

    # Remove the temporary file
    os.remove(temp_filename)

    # Verify the image dimensions
    print(f"Saved image size: {resized_image.size}")  # Should print (64, 64)

    print(f"Saved geometry for {rgi_id} as an image to {output_filename}")

Saved image size: (64, 64)
Saved geometry for RGI60-01.08989 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.08989.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10006 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.10006.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10196 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.10196.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10557 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.10557.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10575 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.10575.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10612 as an image to C:/Users/damsg/OGGM/glacier_geometries_images_small/RGI60-01.10612.png
Saved image size: (64, 64)
Saved geometry for RGI60-01.10621 as an image to C:/Users/damsg/OGGM/glacier_geometries_ima

In [41]:
from PIL import Image
import os

# Directory paths
input_dir = "C:/Users/damsg/OGGM/glacier_geometries_images_small/"
output_dir = "C:/Users/damsg/OGGM/glacier_geometries_images_small_rotmir/"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the rotation angles
# angles = [60, 120, 180, 240, 300, 360]
angles = [90, 180, 270, 360]

# Get all image filenames from the input directory
image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Process each image
for image_file in image_files:
    # Open an image file
    with Image.open(os.path.join(input_dir, image_file)) as img:
        # Process each angle
        for angle in angles:
            # Rotate the image
            rotated = img.rotate(angle, expand=True)
            # Save the rotated image
            rotated.save(os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_rotated_{angle}.png"))

            # Mirror the image
            mirrored = img.transpose(Image.FLIP_LEFT_RIGHT)
            # Save the mirrored image
            # mirrored.save(os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_mirrored.png"))

            # Rotate the mirrored image
            mirrored_rotated = mirrored.rotate(angle, expand=True)
            # Save the mirrored rotated image
            mirrored_rotated.save(os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_mirrored_rotated_{angle}.png"))
